In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from itertools import combinations
from collections import Counter, defaultdict
from astropy.table import Table, join, vstack
from astropy.coordinates import SkyCoord, Distance
from SAGA.utils import add_skycoord
from easyquery import Query, QueryMaker
from astropy.io import ascii

import SAGA
from SAGA.database import FitsTable, GoogleSheets
from SAGA import ObjectCuts as C
from SAGA.utils import add_skycoord, fill_values_by_query
from SAGA.utils.distance import z2v, d2m, m2d
from SAGA.utils.display import show_images
from SAGA.objects.object_catalog import calc_fiducial_p_sat, calc_fiducial_p_sat_corrected

from matplotlib.transforms import Bbox
from scipy.stats import spearmanr

print(SAGA.__version__)

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
def annotate_base(base):
    base["Mr_"] = base["r_mag"] - d2m(base["HOST_DIST"])
    base["Mr"] = np.where(np.isfinite(base["Mr"]) & C.has_spec.mask(base), base["Mr"], base["Mr_"])
    base["log_sm"] = np.where(np.isfinite(base["log_sm"]), base["log_sm"], 1.254 + 1.0976 * base["gr"] - 0.4 * base["Mr"])
    
    base["human_selected"] = 0
    hs = saga.database["human_selected"].read()
    for i in range(1, 4):
        base["human_selected"] += np.in1d(base["OBJID"], Query(f"score == {i}").filter(hs, "OBJID")).astype(np.int) * i

    return base

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Documents/Research/SAGA/PaperII/data-archive/saga_base_all.fits").read()
base = saga.host_catalog.construct_host_query("paper2_complete").filter(base)
assert len(np.unique(base["HOSTID"])) == 36

base = annotate_base(base)

In [ ]:
nsat_missing_all = int(np.ceil(Query(~C.has_spec, C.faint_end_limit).filter(base, "p_sat_corrected").sum()))
nsat_missing_limit = int(np.ceil(Query(~C.has_spec, C.r_abs_limit).filter(base, "p_sat_corrected").sum()))
print(nsat_missing_all, nsat_missing_limit)

In [ ]:
sats = C.is_sat.filter(base)
sats_maybe_all = Query(~C.has_spec, "p_sat_corrected > 0").filter(base)

In [ ]:
seed = 1177 #923 #444 #177 # 114 # 47
while True:
    #seed += 1
    sats_maybe = Query(
        (lambda p: p > np.random.RandomState(seed).rand(len(p)), "p_sat_corrected"),  # must put in 1st place
        ~C.has_spec, 
    ).filter(base)

    if (np.abs((C.faint_end_limit.count(sats_maybe) - nsat_missing_all) < 1.5 and
        C.r_abs_limit.count(sats_maybe)) == nsat_missing_limit and
        (~Query(C.high_priority_sb_tight, C.gr_cut_tight, "Mr > -15.5")).count(sats_maybe) == 0):
        print(seed)
        break
    else:
        seed += 1
        
print(C.r_abs_limit.count(sats_maybe), len(sats_maybe))

In [ ]:
assert len(sats) == 127
assert C.r_abs_limit.count(sats) == 123
assert Query("EW_Halpha < 2").count(sats) == 18
assert Query("EW_Halpha < 2", C.r_abs_limit).count(sats) == 18
assert Query("EW_Halpha < 2", "Mr < -16").count(sats) == 2
assert len(np.unique(sats["HOSTID"])) == 34

In [ ]:
is_p1_sats = Query(C.is_sat, saga.host_catalog.construct_host_query("paper1_complete"), QueryMaker.in1d("OBJID", [4443380000000000475, 3115540000000003218], invert=True))

In [ ]:
import matplotlib as mpl

EW_Halpha_map = mpl.colors.LinearSegmentedColormap.from_list(
    'EW_Halpha_map',
    np.vstack((plt.cm.Blues_r(np.linspace(0, 0.75, 256)), plt.cm.Reds(np.linspace(0.25, 1, 256))))[::-1],
)

def EW_Halpha_normalize(x):
    x = np.asarray(x)
    with np.errstate(all="ignore"):
        return np.where(x > 2, np.log2(x)*2, x)
    
EW_Halpha_norm = mpl.colors.TwoSlopeNorm(EW_Halpha_normalize(2), EW_Halpha_normalize(-2), EW_Halpha_normalize(200))

In [ ]:
def print_sci(x, n=1):
    x = float(x)
    s = (f"{{:.{n}E}}").format(x)
    a, _, b = s.partition("E")
    if abs(int(b)) <= 1:
        return (f"{{:.{n}f}}").format(x)
    b = int(b)
    return f"{a} \\times 10^{{{b}}}"

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10.5, 6.5), gridspec_kw={'hspace': 0.35, 'wspace':0.3})

EW_Halpha_ticks = [-2, 0, 2, 5, 10, 20, 50, 100, 200]
EW_Halpha_ticklabels = [f"$\\leq {t}$" if t == EW_Halpha_ticks[0] else (f"$\\geq {t}$" if t == EW_Halpha_ticks[-1] else str(t)) for t in EW_Halpha_ticks]

####
DO_PAPER_I_ONLY = False #or True
####

sats_this = is_p1_sats.filter(sats) if DO_PAPER_I_ONLY else sats
sats_this.sort("EW_Halpha", reverse=True)

#####
ax_this = ax[0,0]

CS = ax_this.scatter(sats_this["Mr"], sats_this["gr"], c=EW_Halpha_normalize(sats_this["EW_Halpha"]), cmap=EW_Halpha_map, norm=EW_Halpha_norm, alpha=0.9, rasterized=True, label="Confirmed")
if not DO_PAPER_I_ONLY:
    ax_this.scatter(sats_maybe["Mr"], sats_maybe["gr"], marker='o', c='None', edgecolors='k', lw=0.5, label="Unconfirmed")

mr = np.linspace(-20, -12, 11)

ax_this.set_xlim(-20.5, -11.5)
ax_this.set_ylim(-0.05, 0.8)
ax_this.set_xlabel("$M_{r,o}$")
ax_this.set_ylabel("$(g-r)_o$")
ax_this.axvspan(-12.3, -11.5, color="grey", alpha=0.4)
ax_this.legend(loc="lower left", handletextpad=0.1, fontsize=12, borderpad=0.2, handlelength=1, frameon=True)

s = spearmanr(sats_this["Mr"], sats_this["gr"])
#ax_this.text(0.4, 0.15, "$(\\rho_s, p) = ({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue)), fontsize=11, transform=ax_this.transAxes)
print("$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue)))
s = spearmanr(np.hstack([sats_this["Mr"], sats_maybe["Mr"]]), np.hstack([sats_this["gr"], sats_maybe["gr"]]))
#ax_this.text(0.4, 0.05, "$(\\rho_s, p) = ({:.3f}, {})$".format(s.correlation, print_sci(s.pvalue)), fontsize=11, transform=ax_this.transAxes)
print("$({:.3f}, {})$".format(s.correlation, print_sci(s.pvalue)))

#####
ax_this = ax[1,0]

CS = ax_this.scatter(sats_this["Mr"], sats_this["sb_r"], c=EW_Halpha_normalize(sats_this["EW_Halpha"]), cmap=EW_Halpha_map, norm=EW_Halpha_norm, alpha=0.9, rasterized=True)
if not DO_PAPER_I_ONLY:
    ax_this.scatter(sats_maybe["Mr"], sats_maybe["sb_r"], marker='o', c='None', edgecolors='k', lw=0.5)

ax_this.set_xlim(-20.5, -11.5)
ax_this.set_ylim(19, 26.5)
ax_this.set_xlabel("$M_{r,o}$")
ax_this.set_ylabel("$\\mu_{r_o, \\mathrm{eff}}$ [mag arcsec$^{-2}$]")
ax_this.axvspan(-12.3, -11.5, color="grey", alpha=0.4)

s = spearmanr(sats_this["Mr"], sats_this["sb_r"])
#ax_this.text(0.57, 0.15, "$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue)), fontsize=11, transform=ax_this.transAxes)
print("$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue)))
s = spearmanr(np.hstack([sats_this["Mr"], sats_maybe["Mr"]]), np.hstack([sats_this["sb_r"], sats_maybe["sb_r"]]))
#ax_this.text(0.57, 0.05, "$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue)), fontsize=11, transform=ax_this.transAxes)
print("$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue)))


####
ax_this = ax[1,1]

CS = ax_this.scatter(sats_this["RHOST_KPC"], sats_this["Mr"], c=EW_Halpha_normalize(sats_this["EW_Halpha"]), cmap=EW_Halpha_map, norm=EW_Halpha_norm, alpha=0.9, rasterized=True)
if not DO_PAPER_I_ONLY:
    ax_this.scatter(sats_maybe["RHOST_KPC"], sats_maybe["Mr"], marker='o', c='None', edgecolors='k', lw=0.5)

ax_this.set_ylim(-11.5, -20.5)
ax_this.set_xlim(-10, 310)
ax_this.set_ylabel("$M_{r,o}$")
ax_this.set_xlabel("Projected distance [kpc]")
ax_this.axhspan(-12.3, -11.5, color="grey", alpha=0.4, rasterized=True)

s = spearmanr(sats_this["RHOST_KPC"], sats_this["Mr"])
#ax_this.text(0.98, 0.92, "$({:.3f}, {})$".format(s.correlation, print_sci(s.pvalue, 2)), fontsize=11, transform=ax_this.transAxes, ha="right")
print("$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue, 2)))

s = spearmanr(np.hstack([sats_this["RHOST_KPC"], sats_maybe["RHOST_KPC"]]), np.hstack([sats_this["Mr"], sats_maybe["Mr"]]))
#ax_this.text(0.98, 0.82, "$({:.3f}, {})$".format(s.correlation, print_sci(s.pvalue, 2)), fontsize=11, transform=ax_this.transAxes, ha="right")
print("$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue, 2)))


####

####
ax_this = ax[0,1]

sats_this1 = C.r_abs_limit.filter(sats_this)
sats_maybe1 = C.r_abs_limit.filter(sats_maybe)

CS = ax_this.scatter(sats_this1["RHOST_KPC"], sats_this1["gr"], c=EW_Halpha_normalize(sats_this1["EW_Halpha"]), cmap=EW_Halpha_map, norm=EW_Halpha_norm, alpha=0.9, rasterized=True)
if not DO_PAPER_I_ONLY:
    ax_this.scatter(sats_maybe1["RHOST_KPC"], sats_maybe1["gr"], marker='o', c='None', edgecolors='k', lw=0.5)

ax_this.set_xlim(-10, 310)
ax_this.set_ylim(-0.05, 0.8)
ax_this.set_xlabel("Projected distance [kpc]")
ax_this.set_ylabel("$(g-r)_o$")
ax_this.axhspan(-12.3, -11.5, color="grey", alpha=0.4, rasterized=True)
ax_this.text(0, 0, "($M_{r,o} < -12.3$ only)", fontsize=12)

s = spearmanr(sats_this["RHOST_KPC"], sats_this["gr"])
#ax_this.text(0.98, 0.15, "$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue)), fontsize=11, transform=ax_this.transAxes, ha="right")
print("$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue, 2)))

s = spearmanr(np.hstack([sats_this["RHOST_KPC"], sats_maybe["RHOST_KPC"]]), np.hstack([sats_this["gr"], sats_maybe["gr"]]))
#ax_this.text(0.98, 0.05, "$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue)), fontsize=11, transform=ax_this.transAxes, ha="right")
print("$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue, 2)))


####

cbar = plt.colorbar(CS, ax=ax, ticks=EW_Halpha_normalize(EW_Halpha_ticks), fraction=0.02, aspect=50, pad=0.03, shrink=0.7)
cbar.ax.set_rasterized(True)
cbar.ax.set_yticklabels(EW_Halpha_ticklabels)
cbar.solids.set_edgecolor("face")
cbar.ax.minorticks_off()
cbar.ax.set_ylabel("EW(H$\\alpha$) [$\\mathrm{\\AA}$]", fontsize=14, labelpad=-10)

if not DO_PAPER_I_ONLY:
    plt.savefig("/home/yymao/Downloads/halpha.pdf", dpi=200)

In [ ]:
Query('EW_Halpha < 2', "Mr < -16").filter(sats, ["OBJID", "RA", "DEC", "HOSTID", "MASKNAME", "r_mag"])

# sb--radius-color distribution

In [ ]:
mw = SAGA.database.GoogleSheets("1O8tGgnHXRcAT8P78J3V2pWVb3CWE1Hvcowily79tdg4", 0).read()
mw["gr"] = mw["Mg_o"] - mw["Mr_o"]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3.6))

EW_Halpha_ticks = [-2, 0, 2, 5, 10, 20, 50, 100, 200]
EW_Halpha_ticklabels = [f"$\\leq {t}$" if t == EW_Halpha_ticks[0] else (f"$\\geq {t}$" if t == EW_Halpha_ticks[-1] else str(t)) for t in EW_Halpha_ticks]

sats.sort("log_sm", reverse=True)

#####
ax_this = ax[0]

ax_this.scatter(sats_maybe["radius"]*sats_maybe["HOST_DIST"]*1000/3600/180*np.pi, sats_maybe["sb_r"], marker='o', c='None', edgecolors='k', lw=0.6)
CS = ax_this.scatter(sats["radius"]*sats["HOST_DIST"]*1000/3600/180*np.pi, sats["sb_r"], c=sats["log_sm"], vmin=5.5, vmax=10, alpha=0.95, rasterized=True, label="SAGA")
ax_this.scatter(mw["reff_kpc"], mw["sb_r_eff"], marker='*', c=mw["log_sm"], vmin=5.5, vmax=10, lw=0.6, edgecolors=plt.cm.gray(0.2), s=50, alpha=1, rasterized=True, label="MW")

ax_this.set_xscale("log")
ax_this.set_xlim(0.1, 10)
ax_this.set_ylim(19, 26.5)
ax_this.set_xlabel("$R_{r, \\mathrm{eff}}$ [kpc]")
ax_this.set_ylabel("$\\mu_{r_o, \\mathrm{eff}}$ [mag arcsec$^{-2}$]")
ax_this.legend(loc="lower right", handletextpad=0.1, fontsize=13, borderpad=0.25, handlelength=1, frameon=True, markerfirst=False)

y0, y1 = ax_this.get_ylim()
x0, x1 = ax_this.get_xlim()
ax_this.axvline(1.5, ymin=(24.7-y0)/(y1-y0), color=plt.cm.gray(0.4), zorder=-99, ls=":")
ax_this.axhline(24.7, xmin=np.log(1.5/x0)/np.log(x1/x0), color=plt.cm.gray(0.4), zorder=-99, ls=":")
ax_this.text(1.5*1.05, 26.1, "UDGs", fontsize=11)

s = spearmanr(sats["radius"]*sats["HOST_DIST"], sats["sb_r"])
print("$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue, 2)))

s = spearmanr(np.hstack([sats["radius"]*sats["HOST_DIST"], sats_maybe["radius"]*sats_maybe["HOST_DIST"]]), np.hstack([sats["sb_r"], sats_maybe["sb_r"]]))
print("$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue, 2)))


#####
ax_this = ax[1]

ax_this.scatter(sats_maybe["gr"], sats_maybe["sb_r"], marker='o', c='None', edgecolors='k', lw=0.5)
CS = ax_this.scatter(sats["gr"], sats["sb_r"], c=sats["log_sm"], vmin=5.5, vmax=10, alpha=0.95, rasterized=True)
ax_this.scatter(mw["gr"], mw["sb_r_eff"], marker='*', c=mw["log_sm"], vmin=5.5, vmax=10, lw=0.5, edgecolors=plt.cm.gray(0.2), s=50, alpha=0.95, rasterized=True, zorder=99)

ax_this.set_xlim(0, 0.8)
ax_this.set_ylim(19, 26.5)
ax_this.set_xlabel("$(g-r)_o$")
ax_this.set_ylabel("$\\mu_{r_o, \\mathrm{eff}}$ [mag arcsec$^{-2}$]")


s = spearmanr(sats["gr"], sats["sb_r"])
print("$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue, 2)))

s = spearmanr(np.hstack([sats["gr"], sats_maybe["gr"]]), np.hstack([sats["sb_r"], sats_maybe["sb_r"]]))
print("$({:.2f}, {})$".format(s.correlation, print_sci(s.pvalue, 2)))


fig.tight_layout()
cbar = plt.colorbar(CS, ax=ax, pad=0.02)
cbar.ax.set_ylabel("$\\log\;[M_*/M_\\odot]$", labelpad=0, fontsize=13)
cbar.ax.set_rasterized(True)

plt.savefig("/home/yymao/Downloads/sb-radius-color.pdf", dpi=200)

# Quenched fraction

In [ ]:
fq_lg = ascii.read(format="fast_tab", names=["log_sm", "fq", "fq_ue", "fq_le"], data_start=0,
    table="""3.49	1.00	1.00	0.77
4.49	1.00	1.00	0.90
5.49	0.94	0.96	0.83
6.49	1.00	1.00	0.77
7.49	0.80	0.88	0.54
8.49	0.66	0.81	0.38
9.18	0.00	0.60	0.00
""")

fq_tinker = ascii.read(format="fast_tab", names=["log_sm0", "fq", "log_sm1",  "fq_ue", "log_sm2", "fq_le"], data_start=0,
    table="""9.7632	0.0625	9.7655	0.0455	9.7633	0.0788
9.8511	0.1149	9.8510	0.0986	9.8511	0.1312
9.9459	0.1340	9.9459	0.1170	9.9459	0.1503
10.0479	0.1835	10.0479	0.1637	10.0480	0.2019
10.1477	0.2514	10.1476	0.2295	10.1477	0.2734
10.2449	0.2790	10.2472	0.2563	10.2473	0.3010
10.3494	0.3618	10.3493	0.3363	10.3495	0.3859
10.4468	0.4433	10.4467	0.4142	10.4445	0.4709
10.5488	0.4666	10.5487	0.4340	10.5489	0.4985
10.6485	0.5381	10.6484	0.5033	10.6486	0.5728
10.7458	0.5812	10.7481	0.5437	10.7483	0.6188
10.8503	0.6471	10.8501	0.5982	10.8504	0.6938
10.9523	0.7079	10.9522	0.6526	10.9525	0.7632
11.0495	0.7249	11.0493	0.6575	11.0521	0.7929
11.1397	0.7517	11.1394	0.6695	11.1399	0.8346
11.2347	0.8211	11.2319	0.6943	11.2327	0.9472
""")
fq_tinker["log_sm"] = (fq_tinker["log_sm0"] + fq_tinker["log_sm1"] + fq_tinker["log_sm2"]) / 3

fq_geha = ascii.read(format="fast_tab", names=["log_sm", "fq","fq_e"], data_start=0,
    table="""9.9	0.084	0.0052
9.7	0.031	0.0032
9.5	0.017	0.0023
9.3	0.007	0.0016
9.1	0.002	0.0014
8.9	0.000	0.0019
8.7	0.000	0.0028
8.5	0.000	0.0043
8.3	0.000	0.0065
8.1	0.000	0.0091
7.8	0.000	0.0093
7.3	0.000	0.0193
""")
fq_geha["fq_ue"] = fq_geha["fq"] + fq_geha["fq_e"]
fq_geha["fq_le"] = np.maximum(0, fq_geha["fq"] - fq_geha["fq_e"])

In [ ]:
plt.figure(figsize=(4.8, 3.6))

plt.fill_between(fq_tinker["log_sm"], fq_tinker["fq_le"], fq_tinker["fq_ue"], color="C4", alpha=0.7, lw=0, rasterized=True)
plt.fill_between(fq_geha["log_sm"], fq_geha["fq_le"], fq_geha["fq_ue"], color="C4", alpha=0.7, rasterized=True)


plt.errorbar(fq_lg["log_sm"], fq_lg["fq"], marker='x', ls="", c="C1", ms=6)
plt.fill_between(fq_lg["log_sm"], fq_lg["fq_le"], fq_lg["fq_ue"], color="C1", alpha=0.25, lw=0, rasterized=True)

pbins = np.linspace(0, 100, 8)
log_sm = np.concatenate([sats['log_sm']])
bins = np.percentile(log_sm, pbins)
x = np.percentile(log_sm, (pbins[1:]+pbins[:-1])*0.5)

interloper_corr = lambda d: 0.95 - d*0.001

n_q = np.histogram(Query('EW_Halpha < 2').filter(sats, 'log_sm'), bins)[0]
sats_q = Query('EW_Halpha < 2').filter(sats, ["log_sm", "RHOST_KPC"])
sats_maybe_q = Query().filter(sats_maybe_all, ["log_sm", "RHOST_KPC", "p_sat_corrected"])
n_q_corr = np.histogram(sats_q['log_sm'], bins, weights=interloper_corr(sats_q["RHOST_KPC"]))[0]
n_q_corr += np.histogram(sats_maybe_q['log_sm'], bins, weights=sats_maybe_q["p_sat_corrected"]*interloper_corr(sats_maybe_q["RHOST_KPC"]))[0]

n_all = np.histogram(sats['log_sm'], bins)[0]
n_all_corr = np.histogram(sats['log_sm'], bins, weights=interloper_corr(sats["RHOST_KPC"]))[0]
n_all_corr += np.histogram(sats_maybe_all['log_sm'], bins, weights=sats_maybe_all["p_sat_corrected"]*interloper_corr(sats_maybe_all["RHOST_KPC"]))[0]

p = n_q/n_all
pb = (n_q+1)/(n_all+2)
perr = np.sqrt(pb*(1-pb) / n_all)
p1 = np.minimum(1, p + perr)
p2 = np.maximum(0, p - perr)
p_corr = np.maximum(0, (n_q_corr/n_all_corr) - p)

plt.errorbar(x, p, yerr=(p-p2, p1-p), ls='', marker='o', lw=2, capsize=2, ms=5, c="C2")
plt.errorbar(x, p, yerr=(np.zeros_like(p), p1-p+p_corr), ls='', marker='', alpha=0.6, c="C2", lw=2, rasterized=True)

names=["m_star", "f_quench", "e_shot_low", "e_shot_high", "e_correction"]
t = Table(np.vstack((x, p, p-p2, p1-p, p_corr)).T, names=names)
for name in names:
    t[name].format = "%.3f"
t.pprint(-1, -1)

this_work_label = "This work" #and "Mao+20"
plt.xlim(5.3, 11)
plt.text(6.75, 0.2, f"SAGA sat.\n({this_work_label})", ha="right", va="bottom", color="C2", fontsize=13, fontweight="bold")
plt.text(7.2, 0.91, "MW+M31 sat. (Wetzel+15)", ha="left", va="bottom", color="C1", fontsize=13, fontweight="bold")
plt.text(10.7, 0.6, "Field gal.\n(Geha+12)", ha="right", va="bottom", color="C4", fontsize=13, fontweight="bold")
#plt.legend(loc="upper right",  markerfirst=False, fontsize=13,frameon=True, handletextpad=0)
plt.xlabel("$\\log\;[M_*/M_\\odot]$")
plt.ylabel("Quenched fraction")
plt.ylim(-0.05, 1.05)
plt.axhline(0, lw=0.5, color="grey")
plt.axhline(1, lw=0.5, color="grey")
plt.tight_layout()

if this_work_label == "This work":
    plt.savefig("/home/yymao/Downloads/f-quench.pdf", bbox_inches='tight', dpi=200)